## Arrays

Julia has provides a first-class multidimensional array implementation, with the optimizations you want/expect for homogeneous element types and the generality you want for heterogeneous types.

In [ ]:
a = [10, 20, 30]

In [ ]:
a = ["foo", "bar", 10]

These call standard library routines that picked the element types of the arrays as `Int64` and `Any`.

### Interpreting the properties of arrays : 

In [ ]:
size(a)

In [ ]:
ndims(a)

As you can see above, the arrays are flat, i.e., there is only a single dimension in which there are 3 elements. We can also specify the type and construct an uninitialised array,

In [ ]:
Array{Float64}(3)

In [ ]:
# To create an empty array of type Float64
v = Float64[]

In [ ]:
push!(v, 123)

In [ ]:
push!(v, "hello")

Or use functions which return special types of arrays, 

In [ ]:
rand(10)

Note that by default rand() returns an Array with elements of type Float64 

In [ ]:
A = rand(4, 4)

In [ ]:
A + 10I

In [ ]:
dump(10.5I)

In [ ]:
[rand(3,4) 0I; 0I rand(3,4)]

In [ ]:
diagm([1,2,3,4])

In [ ]:
d = Diagonal([1,2,3,4])

In [ ]:
d^2

In [ ]:
A + d^2

### Arrays : Vectors or matrices

There are containers of type Vector and Matrix. Let us see what these are, and how they are derived from the general Array type

In [ ]:
Array{Int64, 1} == Vector{Int64}

In [ ]:
Array{Int64, 2} == Matrix{Int64}

In [ ]:
Array{Int64, 1} == Matrix{Int64}

In [ ]:
Array{Int64, 3} == Matrix{Int64}

Changing dimensions of Arrays can be done using the reshape() function,

In [ ]:
a = [1, 2, 3, 4, 5, 6]

In [ ]:
b = reshape(a, 3, 2)

So the rule is if size(a) = k, then reshape(a, m, n) such that m * n = k.

In [ ]:
reshape(a, 2, :)

In [ ]:
squeeze([1 2 3 4], 1)

### Indexing Arrays

In [ ]:
a = rand(3, 3)

The above 3x3 Array appears to be a 2-dimensional array with 9 elements, but its actual representation is a view of a linear contiguous sequence of numbers in memory.

In [ ]:
a = rand(1000,1000)
s1 = zero(eltype(a));
idx = eachindex(a)
@time for i in idx
    s1 += a[i]
end
(m,n)=size(a)

s2 = zero(eltype(a));
@time for i = 1:m, j=1:n
    s1 += a[i, j]
end

s3 = zero(eltype(a));
@time for j=1:n, i=1:m
    s1 += a[i, j]
end

In [ ]:
c = rand(6,4)

In [ ]:
c[1:5, 1:2]

Observe from the previous that Julia follows column-major ordering. Below experiment shows that Julia also stores in column-major order in memory. The time taken for accessing a column is faster than the time taken to access a row.

In [ ]:
d = rand(10000, 10000)

In [ ]:
@time slice = d[1:10000,1];
slice

In [ ]:
@time slice = d[1,1:10000];
slice

### Cartesian vs Linear indexing

Some array types support efficiently indexing via a "linearized" single index. Depending on the representation, other array types may need to convert back and forth between Cartesian `(i,j,...)` index tuples and scalar linear indices. This relies on integer division which is much more expensive than multiplication and addition!

In [ ]:
@show size(a)
a[2500] # linear indexing

In [ ]:
?ind2sub

In [ ]:
?sub2ind

In [ ]:
ind2sub(size(a), 2500)

In [ ]:
sub2ind(size(a), 500, 3)

### Copying Arrays

In [ ]:
a = [1, 2, 3, 4]

In [ ]:
b = a

In [ ]:
b[1] = 10

In [ ]:
a

This also means that the value of an array is its pointer to the memory location. To actually create another copy of an array use the `copy()` function

In [ ]:
c = copy(a)

In [ ]:
c[1] = 1

In [ ]:
a

Let us go deeper one level, 

In [ ]:
a = Vector{Vector{Float64}}(3)
a[1] = [1;2;3]
a[2] = [1;2]
a[3] = [3;4;5]
a

In [ ]:
b = a

In [ ]:
b[1] = [1, 4, 5]
a

In [ ]:
c = copy(a)

In [ ]:
c[1] = [1,2,3]
a

In [ ]:
c[end][end] = 7

In [ ]:
a

`copy` did not help here, this is because `copy()` is only a shallow copy. Use `deepcopy()` which not only copies the outer structure but also recursively copies the internal structures too.

In [ ]:
d = deepcopy(a)

In [ ]:
d[end][end] = 13

In [ ]:
a

### Operations on Arrays

#### Array methods

In [ ]:
a = [1, -2, 3, 0]

In [ ]:
length(a)

In [ ]:
sum(a)

In [ ]:
mean(a)

In [ ]:
std(a)

In [ ]:
var(a)

In [ ]:
b = sort(a, rev=true)

#### Matrix operations

In [ ]:
a = ones(1, 2)

In [ ]:
b = ones(2, 2)

In [ ]:
a * b

To solve the linear system `A X = B` for `X` use `A \ B`

In [ ]:
A = [1 2; 2 3]

In [ ]:
B = ones(2, 2)

In [ ]:
A \ B

In [ ]:
inv(A) * B

Although the last two operations give the same result, the first one is numerically more stable and should be preferred in most cases

#### Elementwise operations

In [ ]:
ones(2, 2) * ones(2, 2) # This results in regular matrix multiplication

In [ ]:
ones(2, 2) .* ones(2, 2) # This results in elementwise multiplication

In [ ]:
a = [10, 0, 30]

In [ ]:
b = [-100, 0, 100]

In [ ]:
b .> a

In [ ]:
b <= a

In [ ]:
a .== b

In [ ]:
[1,2,3] == [1.,2.,3.]

#### Conditional extraction

In [ ]:
a = randn(4)

In [ ]:
a .< 0

In [ ]:
a[a .< 0]

#### Vectorised function application

In [ ]:
exp.([1,2,3])

In [ ]:
exp.(rand(3,3))

In [ ]:
expm(rand(3,3)) # soon just `exp(rand(3,3))`

In [ ]:
[log(x) for x in 1:3] # Obtain similar results using comprehension

#### Map operation using anonymous functions

In [ ]:
map(x->2x, 1:10)

#### Broadcast

Julia offers broadcast(), which expands singleton dimensions in array arguments to match the corresponding dimension in the other array without using extra memory, and applies the given function elementwise:

In [ ]:
a = rand(2,1)

In [ ]:
A = rand(2,3)

In [ ]:
broadcast(+, a, A)
#@show vec(a)
#@show typeof(vec(a))
#broadcast(+, vec(a), A)

In [ ]:
A = 1:5
B = [2;3;4;5;6]

In [ ]:
C = reverse(B)

In [ ]:
A.*B.*C